In [1]:
!pip install -q --upgrade pip
# !pip install --force-reinstall transformers==4.57.3 accelerate==0.29.3 huggingface-hub==0.36.0
# !pip install --upgrade accelerate
# !pip install transformers==4.57.3 accelerate==1.12.0 huggingface-hub==0.36.0 datasets==2.14.7 peft==0.18.0

!pip install -q transformers accelerate bitsandbytes datasets peft sentencepiece huggingface_hub


In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

print("Torch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())


Torch: 2.9.1+cu128
CUDA: True


In [4]:

from huggingface_hub import login
login(token="REMOVED")


In [5]:
model_name = "google/gemma-2-2b-it"   # MoE checkpoint

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

print("Model loaded.")


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Model loaded.


In [7]:
dataset = load_dataset("roneneldan/TinyStories", split="train[:1%]")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )

tokenized = dataset.map(tokenize, batched=True)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/21197 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./gemma-moe-lora",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=20,
    learning_rate=2e-4,
    logging_steps=20,
    save_steps=200,
    fp16=True,
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
20,1.672700
40,1.509200
60,1.412700
80,1.381200
100,1.358200
120,1.400600
140,1.380400
160,1.394800
180,1.377600
200,1.367000


In [11]:
model.save_pretrained("./gemma-moe-lora")
tokenizer.save_pretrained("./gemma-moe-lora")

print("Model saved!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

## Compare Old vs New

In [26]:
prompt = "The wise dragon said:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    **inputs,
    max_length=150,
    temperature=0.8,
    top_p=0.9,
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Model and tokenizer loaded.
